In [1]:
import cv2
import mediapipe as mp
import numpy as np
import time, os

# 인식할 동작 목록 정의
actions = ['one', 'two', 'three']

# 각 시퀀스의 길이와 각 동작의 지속 시간 정의
seq_length = 30
secs_for_action = 30


# MediaPipe hands 모델 초기화
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(
    max_num_hands=1,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5)

# 웹캠 열기
cap = cv2.VideoCapture(0)

# 데이터셋을 저장할 디렉토리 생성
created_time = int(time.time())
os.makedirs('dataset', exist_ok=True)

# 각 동작에 대한 데이터를 수집하기 위한 루프
while cap.isOpened():
    for idx, action in enumerate(actions):
        data = [] # 현재 동작에 대한 데이터를 저장할 리스트
        
        # 웹캠에서 프레임을 읽어옴
        ret, img = cap.read()

        img = cv2.flip(img, 1) #이미지를 좌우로 뒤집어 거울 효과

        # 수집할 동작을 표시하는 메시지 출력
        cv2.putText(img, f'Waiting for collecting {action.upper()} action...', org=(10, 30), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(255, 255, 255), thickness=2)
        cv2.imshow('img', img)
        cv2.waitKey(3000) # 데이터 수집 시작 전에 3초 대기

        start_time = time.time() # 시작 시간 기록
        
        # 지정된 기간 동안 데이터 수집
        while time.time() - start_time < secs_for_action:
            ret, img = cap.read()

            img = cv2.flip(img, 1) # 이미지를 좌우로 뒤집음
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # 이미지를 RGB로 변환
            result = hands.process(img) # 이미지를 처리하여 손 검출
            img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR) # 이미지를 BRG로 다시 변환
        
            # 이미지에서 손이 검출된 경우
            if result.multi_hand_landmarks is not None:
                for res in result.multi_hand_landmarks:
                    joint = np.zeros((21, 4)) # 관절 좌표를 저장할 배열 초기화
                    for j, lm in enumerate(res.landmark):
                        joint[j] = [lm.x, lm.y, lm.z, lm.visibility]

                    # 관절 간의 각도 계산
                    v1 = joint[[0,1,2,3,0,5,6,7,0,9,10,11,0,13,14,15,0,17,18,19], :3] # Parent joint
                    v2 = joint[[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20], :3] # Child joint
                    v = v2 - v1 # [20, 3] # 관절 간의 벡터
                    v = v / np.linalg.norm(v, axis=1)[:, np.newaxis] # 벡터 정규화

                    # 내적의 아크 코사인을 사용하여 각도 계산
                    angle = np.arccos(np.einsum('nt,nt->n',
                        v[[0,1,2,4,5,6,8,9,10,12,13,14,16,17,18],:], 
                        v[[1,2,3,5,6,7,9,10,11,13,14,15,17,18,19],:])) # [15,]
                                        
                    angle = np.degrees(angle) # 라디안을 각도로 변환
                    
                    # 각도와 동작 인덱스를 데이터에 추가
                    angle_label = np.array([angle], dtype=np.float32)
                    angle_label = np.append(angle_label, idx)

                    d = np.concatenate([joint.flatten(), angle_label])

                    data.append(d)
                    
                    # 이미지에 손 랜드마크 그리기
                    mp_drawing.draw_landmarks(img, res, mp_hands.HAND_CONNECTIONS)
            
            # 손 랜드마크가 그려진 이미지 표시
            cv2.imshow('img', img)
            if cv2.waitKey(1) == ord('q'): # 'q' 키를 누르면 종료
                break

        # 원시 데이터 저장
        data = np.array(data)
        print(action, data.shape)
        np.save(os.path.join('dataset', f'raw_{action}_{created_time}'), data)

        # 원시 데이터에서 시퀀스 데이터 생성
        full_seq_data = []
        for seq in range(len(data) - seq_length):
            full_seq_data.append(data[seq:seq + seq_length])

        full_seq_data = np.array(full_seq_data)
        print(action, full_seq_data.shape)
        np.save(os.path.join('dataset', f'seq_{action}_{created_time}'), full_seq_data)
    break # 모든 동작에 대한 데이터 수집 후 while 루프 종료
    
# 웹캠 해제 및 모든 OpenCV 창 닫기
cap.release()
cv2.destroyAllWindows()

one (790, 100)
one (760, 30, 100)
two (830, 100)
two (800, 30, 100)
three (840, 100)
three (810, 30, 100)
